In [1]:
!wget https://raw.githubusercontent.com/hsiangjenli/2023-summer-internship/master/crawler/ptt.py
!pip install transformers

--2023-08-29 07:42:34--  https://raw.githubusercontent.com/hsiangjenli/2023-summer-internship/master/crawler/ptt.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7312 (7.1K) [text/plain]
Saving to: ‘ptt.py’

ptt.py              100%[===================>]   7.14K  --.-KB/s    in 0s      

2023-08-29 07:42:35 (52.0 MB/s) - ‘ptt.py’ saved [7312/7312]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.6 MB/s eta 0:00:00


In [2]:
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
import torch

__SENTIMENT_MODEL__ = "IDEA-CCNL/Erlangshen-Roberta-330M-Sentiment"

tokenizer = BertTokenizer.from_pretrained(__SENTIMENT_MODEL__)
model = BertForSequenceClassification.from_pretrained(__SENTIMENT_MODEL__)

In [28]:
from ptt import PTT
import re

# @markdown > 關鍵字請用 `,` 隔開
SCSB_KEYWORDS = "\u4E0A\u6D77, \u5C0F\u5C0F\u5175, \u4E00\u8DEF\u901A, \u638C\u4E0A\u9280, \u53F0\u7063pay"# @param {type:"string"}
SCSB_KEYWORDS = [kw.strip().lower() for kw in SCSB_KEYWORDS.split(",")]
target = 'MobilePay' # @param ["Finance", "creditcard", "Bank_Service", "MobilePay", "Stock"] {allow-input: true}
pages = 15 # @param {type:"slider", min:1, max:30, step:1}

In [29]:
def find_kw(text, kws):
  return re.findall(pattern="|".join(kws), string=text)

info = '\n------\n{title}\n{link}'
emo_info = '..{emo} {r:.4f} {text}'

In [30]:
ptt = PTT(board=target, crawler_pages=pages, sleep=0.1)

In [31]:
for article in ptt.get():
  content = article["content"]
  comments = "\n".join(article["comments"])

  if find_kw(content.lower(), SCSB_KEYWORDS):
    output = model(torch.tensor([tokenizer.encode(content[:512])]))
    r = torch.nn.functional.softmax(output.logits,dim=-1).detach().tolist()[0][1]
    emo = "🎃" if r > 0.5 else "💀"
    print(info.format(title=article["title"], link=article["link"]))
    print(emo_info.format(emo=emo, text="", r=r))

  elif find_kw(comments.lower(), SCSB_KEYWORDS):
    print(info.format(title=article["title"], link=article["link"]))
    for comment in article["comments"]:
      if find_kw(comment.lower(), SCSB_KEYWORDS):
        output = model(torch.tensor([tokenizer.encode(comment)]))
        r = torch.nn.functional.softmax(output.logits,dim=-1).detach().tolist()[0][1]
        emo = "🎃" if r > 0.5 else "💀"
        print(emo_info.format(emo=emo, text=comment, r=r))


------
Re:  超商刷卡/支付/自帶會員整理
https://www.ptt.cc/bbs/MobilePay/M.1643896282.A.C12.html
..🎃 0.9998 

------
Re:  請問台新pay到7-11繳學費
https://www.ptt.cc/bbs/MobilePay/M.1690890728.A.91E.html
..🎃 0.9860 → ChampionYe: 我的上海商銀代收也是不能繳，後來只好都用轉帳，感 08/01 23:33

------
支付寶開放綁信用卡　中港澳旅遊消費不卡
https://www.ptt.cc/bbs/MobilePay/M.1691112943.A.339.html
..🎃 0.9999 

------
Re:  全盈+PAY首綁兆豐MegaLite數位帳戶活動
https://www.ptt.cc/bbs/MobilePay/M.1686903955.A.926.html
..💀 0.0061 → PTTis5566: 上海商銀的不能臨櫃提權，只能結清再來一次 06/20 00:12
..💀 0.0627 推 rtwodtwo: 上海能臨櫃提權啊 而且上海也沒開放自然人憑證申辦 06/20 02:29
..💀 0.0001 推 rtwodtwo: 上海用信用卡驗證 就算臨櫃提到1類也沒用 只能砍掉重來 06/20 02:33

------
華南銀行X全支付 消費回饋最高50%
https://www.ptt.cc/bbs/MobilePay/M.1687189946.A.531.html
..🎃 0.9999 推 capacitor: 上海商銀2000名到現在都沒滿，華南不知會不會撐更久 06/20 07:57
..🎃 0.9992 → sydwuz: 所以之前的台企上海到底是…？ 06/20 10:04
..🎃 0.9995 推 arashivivian: 社群回報上海的到現在也還有啊 06/20 18:43
..🎃 0.9977 → Kazamatsuri: 倒是上海有額滿公告了 XD 06/25 23:32

------
上海銀行信用卡支援Apple Pay滿千送百
https://www.ptt.cc/bbs/MobilePay/M.1687276909.A.C89